# Time in Range Collection

## Loading relevant libraries

`tidyverse` is an R library that contains core packages used to read, analyze, and plot data

In [ ]:
library(tidyverse)
library(lubridate)

## Dexcom

Given that the exported .csv files for Dexcom and FreeStyle Libre are formatted differently, reading and wrangling must also be carried out differently. Following this, the formula for calculating TIR is the same for both CGMS.

### Defining Function

In [ ]:
dexcom <- function(file) {
    suppressWarnings({
        read <- read_csv(file)
        
        # renaming columns for ease of use
        names(read)[8] = 'Glucose_Value'
        names(read)[2] = 'Timestamp'
        names(read)[14] = 'Transmitter_ID'
        names(read)[5] = 'Patient_Info'
        
        # creating string based on username and birthdate for ID so comparisons can be made across timepoints
        info <- filter(read, Patient_Info != 'NA')
        info_string <- info %>% pull(Patient_Info)
        id_string <- paste(info_string, collapse = '')
        id_string
        
        # removing top 11 rows that contain user's name and alert types
        rows <- filter(read, Transmitter_ID != 'NA') %>%
                filter(Glucose_Value != 'NA')
        
        # selecting only timestamp and glucose value columns for calculations
        cols <- select(rows, Timestamp, Glucose_Value)
        
        # pulling vectors from columns
        glucose_vector_str <- cols %>% pull(Glucose_Value)
        glucose_vector <- as.double(glucose_vector_str) # converting to decimals
        
        time_vector <- cols %>% pull(Timestamp)
        minutes <- minutes(time_vector) # number of readings in min
        
        # calculating percentage of sensor readings within 14 days
        # only data from participants with >= 70% will be used
        Sensor <- (length(glucose_vector) / 4032) * 100
        
        # calculations
        Mean <- mean(glucose_vector, na.rm=TRUE) # average glucose
        GMI <- 12.71 + 4.70587 * Mean # glucose management indicator (mmol/mol)
        SD <- sd(glucose_vector, na.rm=TRUE) # standard deviation
        CV <- SD / Mean # coefficient of variation
        
        # time in range
        in_range <- sum(glucose_vector >= 3.9 & glucose_vector <= 10.0, na.rm=TRUE)
        TIR <- round(in_range * 1000 / length(minutes)) / 10
        
        # very low
        very_low <- sum(glucose_vector < 3.0, na.rm=TRUE)
        TBR_VL <- round(very_low * 1000 / length(minutes)) / 10
        
        # low
        low <- sum(glucose_vector >= 3.0 & glucose_vector <= 3.8, na.rm=TRUE)
        TBR_L <- round(low * 1000 / length(minutes)) / 10
        
        # high
        high <- sum(glucose_vector >= 10.1 & glucose_vector <= 13.9, na.rm=TRUE)
        TAR_H <- round(high * 1000 / length(minutes)) / 10
        
        # very high
        very_high <- sum(glucose_vector > 13.9, na.rm=TRUE)
        TAR_VH <- round(very_high * 1000 / length(minutes)) / 10
        
        # consolidating relevant values into data frame
        df <- data.frame(id_string, TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor)
    })
}

### Applying Function - Waitlist Participants

Calling the `dexcom` function on all .csv files in the Dexcom folder to create a data frame with all data from all participants in the waitlist group who use Dexcom

In the final data frame and .csv file (adding both Dexcom and Freestyle Libre using participants to a single data frame), the participants' `id_string`, i.e. their name and birthday, will not be visible and they will be assigned to a unique ID.

In [ ]:
dex_waitlist <- list.files(path = 'data_final/Dexcom',    
                       pattern = '*.csv', full.names = TRUE) %>% 
  lapply(dexcom) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)
  
dex_waitlist

dex_waitlist_output <- 'dex_waitlist.csv'
write.csv(dex_waitlist, dex_waitlist_output)

### Applying Function - Treatment Participants

Calling the `dexcom` function on all .csv files in the Dexcom folder to create a data frame with all data from all participants in the treatment group who use Dexcom

In the final data frame and .csv file (adding both Dexcom and Freestyle Libre using participants to a single data frame), the participants' `id_string`, i.e. their name and birthday, will not be visible and they will be assigned to a unique ID.

In [ ]:
dex_treatment <- list.files(path = 'data_final/Dexcom',    
                       pattern = '*.csv', full.names = TRUE) %>% 
  lapply(dexcom) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)
  
dex_treatment

dex_treatment_output <- 'dex_treatment.csv'
write.csv(dex_treatment, dex_treatment_output)

### Applying Function - Peer Supporters

Calling the `dexcom` function on all .csv files in the Dexcom folder to create a data frame with all data from all peer supporters who use Dexcom

In the final data frame and .csv file (adding both Dexcom and Freestyle Libre using participants to a single data frame), the peer supporters' `id_string`, i.e. their name and birthday, will not be visible and they will be assigned to a unique ID.

In [ ]:
dex_ps <- list.files(path = 'data_final/Dexcom',    
                       pattern = '*.csv', full.names = TRUE) %>% 
  lapply(dexcom) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)
dex_ps

dex_ps_output <- 'dex_ps.csv'
write.csv(dex_ps, dex_ps_output)

## Freestyle Libre 2

### Defining Function

In [ ]:
libre_2 <- function(file) {
    suppressWarnings({
        
        # initial reading to create id
        read_id <- read_csv(file)
        names(read_id)[1] = 'Patient_report'
        names(read_id)[2] = 'Generated_on'
        
        # creating string based on user name and birthdate for id
        info <- filter(read_id, Patient_report != 'FreeStyle Libre 2') %>%
                filter(Patient_report != 'FreeStyle LibreLink') %>%
                filter(Patient_report != 'Device') %>%
                select(Patient_report, Generated_on)
        patient_string <- info %>% pull(Patient_report)
        date_string <- info %>% pull(Generated_on)
        id_string <- paste(patient_string, date_string, collapse = '')
        
        read <- read_csv(file, skip=2)
        
        # renaming columns for ease of modifying
        names(read)[3]='Timestamp'
        names(read)[5]='Glucose_Value'
        
        # removing NA values in Glucose_Value column
        rows <- filter(read, Glucose_Value != 'NA') %>%
                filter(read, Timestamp )
        
        # selecting only timestamp and glucose value columns
        cols <- select(rows, Timestamp, Glucose_Value)
        
        # pulling vectors from columns
        glucose_vector <- cols %>% pull(Glucose_Value)
        
        time_vector <- cols %>% pull(Timestamp)
        time_dttm <- ymd_hms(time_vector)
        minutes <- minutes(time_dttm) # pulling the number of readings (in minutes)
        
        # calculating percentage of sensor usage within 14 days
        Sensor <- (length(glucose_vector) / 1344) * 100
        
        # performing calculations
        Mean <- mean(glucose_vector, na.rm=TRUE) # average glucose
        GMI <- 12.71 + 4.70587 * Mean # glucose management indicator (mmol/mol)
        SD <- sd(glucose_vector, na.rm=TRUE) # standard deviation
        CV <- SD / Mean # coefficient of variation
        
        # time in range
        in_range <- sum(glucose_vector >= 3.9 & glucose_vector <= 10.0, na.rm=TRUE)
        TIR <- round(in_range * 1000 / length(minutes)) / 10
        
        # very low
        very_low <- sum(glucose_vector < 3.0, na.rm=TRUE)
        TBR_VL <- round(very_low * 1000 / length(minutes)) / 10
        
        # low
        low <- sum(glucose_vector <= 3.8 & glucose_vector >= 3.0, na.rm=TRUE)
        TBR_L <- round(low * 1000 / length(minutes)) / 10
        
        # high
        high <- sum(glucose_vector >= 10.1 & glucose_vector <= 13.9, na.rm=TRUE)
        TAR_H <- round(high * 1000 / length(minutes)) / 10
        
        # very high
        very_high <- sum(glucose_vector > 13.9, na.rm=TRUE)
        TAR_VH <- round(very_high * 1000 / length(minutes)) / 10
        
        # data frame
        df <- data.frame(id_string, TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor)
        
    })
}

### Applying Function - Waitlist Participants

Calling the `libre_2` function on all .csv files in the Libre2 folder to create a data frame with data from all participants in the waitlist group who use the Freestyle Libre 2.

In [ ]:
libre2_waitlist <- list.files(path = 'data_final/Libre2',    
                       pattern = "*.csv", full.names = TRUE) %>% 
  lapply(libre_2) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)

libre2_waitlist

libre2_waitlist_output <- 'libre2_waitlist.csv'
write.csv(libre2_waitlist, libre2_waitlist_output)

### Applying Function - Treatment Participants

Calling the `libre_2` function on all .csv files in the Libre2 folder to create a data frame with data from all participants in the treatment group who use the Freestyle Libre 2.

In [ ]:
libre2_treatment <- list.files(path = 'data_final/Libre2',    
                       pattern = "*.csv", full.names = TRUE) %>% 
  lapply(libre_2) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)

libre2_treatment

libre2_treatment_output <- 'libre2_treatment.csv'
write.csv(libre2_treatment, libre2_treatment_output)

### Applying Function - Peer Supporters

Calling the `libre_2` function on all .csv files in the Libre2 folder to create a data frame with data from all peer supporters who use the Freestyle Libre 2.

In [ ]:
libre2_ps <- list.files(path = 'data_final/Libre2',    
                       pattern = "*.csv", full.names = TRUE) %>% 
  lapply(libre_2) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)

libre2_ps

libre2_ps_output <- 'libre2_ps.csv'
write.csv(libre2_ps, libre2_ps_output)

## Consolidating Dexcom and Freestyle Libre Data - Waitlist Participants

Binding the data frames from participants who are Dexcom users and Freestyle Libre 2 users in the waitlist group together into a single data frame, which is written into a .csv file that can be downloaded.

In [ ]:
waitlist_output <- 'waitlist.csv'

collection <- rbind(dex_waitlist, libre2_waitlist)

collection_id <- transform(collection, ID = as.numeric(factor(id_string)))

waitlist_collection <- collection_id %>% select(TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor, ID) %>% arrange(ID)
waitlist_collection
write.csv(waitlist_collection, waitlist_output)

## Consolidating Dexcom and Freestyle Libre Data - Treatment Participants

Binding the data frames from participants who are Dexcom users and Freestyle Libre 2 users in the treatment group together into a single data frame, which is written into a .csv file that can be downloaded.

In [ ]:
treatment_output <- 'treatment.csv'

collection <- rbind(dex_treatment, libre2_treatment)

collection_id <- transform(collection, ID = as.numeric(factor(id_string)))

treatment_collection <- collection_id %>% select(TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor, ID) %>% arrange(ID)
treatment_collection
write.csv(treatment_collection, treatment_output)

## Consolidating Dexcom and Freestyle Libre Data - Peer Supporters

Binding the data frames from peer supporters who are Dexcom users and Freestyle Libre 2 users together into a single data frame, which is written into a .csv file that can be downloaded.

In [ ]:
ps_output <- 'ps.csv'

collection <- rbind(dex_ps, libre2_ps)

collection_id <- transform(collection, ID = as.numeric(factor(id_string)))

ps_collection <- collection_id %>% select(TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor, ID) %>% arrange(ID)
ps_collection
write.csv(ps_collection, ps_output)